In [47]:
import os                                                              
import time   
import pickle
import sys
import numpy as np                                                     
import pandas as pd   
import seaborn as sns                                                  
import matplotlib.pyplot as plt
import math
import re
                                                                       
from sklearn.metrics import mean_absolute_error                        
from sklearn.metrics import mean_squared_error                         
from sklearn.model_selection import train_test_split                   
from sklearn.preprocessing import StandardScaler                       
from sklearn.model_selection import train_test_split

import keras                                                           
from keras.optimizers import Adam                                      
from keras.models import Model, Sequential                             
from keras.layers import Input, Dense, Dropout, merge                  
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping 

import ase                                                             
import ase.build                                                       
from ase import Atoms                                                  
from ase.atoms import Atoms                                            
from ase.io import read, write                                         
from ase.calculators.dftb import Dftb                                  
from ase.units import Hartree, mol, kcal, Bohr                         
                                                                       
from src.Calculator import src_nogrd                                                       
from src.Calculator.src_nogrd import sym_func_show   
from src.Calculator.src_nogrd import symmetry_function                                         
from src.Calculator.src_nogrd import xyzArr_generator                                 
from src.Calculator.src_nogrd import feat_scaling_func                                
from src.Calculator.src_nogrd import at_idx_map_generator
from src.Calculator.src_nogrd import at_idx_map_generator_old
from src.Calculator.store_models import write_subnet_text
                                                                                                                           
import pickle
from itertools import combinations_with_replacement as comb_replace
                                                                       
import src.Utils.DirNav                                                
from src.Utils.dftb_traj_io import read_scan_traj
import src.Utils.netBuilder
from src.Utils.netBuilder import netBuilder

In [48]:
import keras
geom_filename          = 'final_Au39.xyz' # I appended your sample structures

md_train_arr_origin    = read_scan_traj(filename=geom_filename)
md_train_arr = md_train_arr_origin.copy(deep=False).reset_index(drop=True)

In [49]:
# Maintainence: Natom is a global variable 
# Assumes that the configuration does not change the Number of atoms. 
nAtoms, xyzArr = xyzArr_generator(md_train_arr)# Calculate distance dataframe from xyz coordinates
distances = src_nogrd.distances_from_xyz(xyzArr, nAtoms)

In [50]:
struct = ase.io.read(geom_filename, format = "xyz")
SUPPORTED_ELEMENTS = ['H','O','Au']

In [51]:
cutoff_rad = 7 #max(distances.max(1))
Rs_array = np.linspace(0.2, 7, num=14) #np.linspace(distances.min(1), max(distances.max(1)), num=int(max(distances.max(1))-min(distances.min(1))))   # based on max and min of the distances
eta_array =  5/(np.square(0.2*Rs_array))  #((max(distances.max(1))- min(distances.min(1))))/(2*np.square(min(distances.min(1))*Rs_array))
rad_params = np.array([(Rs_array[i], eta_array[i], cutoff_rad) for i in range(len(Rs_array)) ])



# angular symmetry function parameters
cutoff_ang =  5 #math.radians(min(all_ang)) #math.radians(max(all_ang))
lambd_array = np.array([-1, 1])
zeta_array = np.array([1, 4, 16]) #zeta_array = np.array([1, 4, 16])
#zeta_array = np.array([math.radians(sum(all_ang)/len(all_ang))])
eta_ang_array = np.array([0.001, 0.01, 0.05]) #eta_ang_array = np.array([0.001, 0.01, 0.05])
#eta_ang_array = np.array([math.radians(min(all_ang))])

# Each of the element need to be parametrized for all of the list. 
angList = np.array([e1+e2 for e1, e2 in comb_replace(SUPPORTED_ELEMENTS, 2)])
#print(angList)
ang_comp = {el : angList for el in SUPPORTED_ELEMENTS}
ang_params = np.array([[eta, zeta, lambd, cutoff_ang] for eta in eta_ang_array for zeta in zeta_array for lambd in lambd_array])

In [52]:
np.array([e1+e2 for e1, e2 in comb_replace(SUPPORTED_ELEMENTS, 2)])

array(['HH', 'HO', 'HAu', 'OO', 'OAu', 'AuAu'], dtype='<U4')

In [53]:
at_idx_map_t = at_idx_map_generator_old(md_train_arr[0])
at_idx_map = {el : at_idx_map_t[el] for el in SUPPORTED_ELEMENTS}

In [54]:
Gparam_dict = {}
for at_type in at_idx_map.keys():
   Gparam_dict[at_type] = {}
   Gparam_dict[at_type]['rad'] = {}
   for at2_rad in at_idx_map.keys():
           Gparam_dict[at_type]['rad'][at2_rad] = rad_params

   # This Section is already designed to be genera
   Gparam_dict[at_type]['ang'] = {}
   for at23_ang in ang_comp[at_type]:
       Gparam_dict[at_type]['ang'][at23_ang] = ang_params
for at_type in Gparam_dict.keys():
   print(Gparam_dict[at_type]['rad'].keys())

dict_keys(['H', 'O', 'Au'])
dict_keys(['H', 'O', 'Au'])
dict_keys(['H', 'O', 'Au'])


In [56]:
Gfunc_data = symmetry_function(distances, at_idx_map, Gparam_dict)

C:\Users\claud\AppData\Local\Temp\ipykernel_24124\800211604.py:79: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Gfunc_data = pd.Series([])
C:\Users\claud\AppData\Local\Temp\ipykernel_24124\800211604.py:87: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Gfunc_data[at_type] = pd.Series([])
C:\Users\claud\AppData\Local\Temp\ipykernel_24124\800211604.py:87: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Gfunc_data[at_type] = pd.Series([])
C:\Users\claud\AppData\Local\Temp\ipykernel_24124\800211604.py:87: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silen

In [57]:
# Glboal Var
n_samples = len(md_train_arr)


# Maintainence // Problem: In case we have no atom for that particular element, 
# it is difficult to give the input dimension
    
# Maintain: I assume that same amount of symmetry function is used for each atom  
# Maintain: Need a input_dim_list_generator function in the future
"""
def input_dim_list_generator(element):
    # In case of element with 0 atoms in the configuration
    nAtomsForEle = len(at_idx_map(element));
    if len nAtomsForEle == 0:
        return (nAtomsForEle, )
        """
#nSymFunc = Gfunc_data['O'][0].shape[1]
n_symm_func = Gfunc_data['O'][  at_idx_map['O'][0]].shape[1]
#input_dim_list = [(len(at_idx_map[at]) ,nSymFunc, nb_feat) for at in at_idx_map.keys()]

input_dim_list = {}
for at_type in at_idx_map:
    input_dim_list[at_type] = (n_symm_func, 1)

In [58]:
todos=np.arange(0,n_samples) #Number of structures given
Feat_train_scaled = {}
for at_type in at_idx_map.keys():
    Feat_train_scaled[at_type] = {}
    for at in at_idx_map[at_type]:
        Feat_train_scaled[at_type][at] = Gfunc_data[at_type][at][todos,]

inp_train = []
for at_type in at_idx_map.keys():

    for atA in at_idx_map[at_type]:
        inp_train.append(Feat_train_scaled[at_type][atA])

In [59]:
def build_subnets(SUPPORTED_ELEMENTS):
    elemental_subnets = {}
    for element in SUPPORTED_ELEMENTS:
        globals()[str(element)+"-subnet"]=keras.models.load_model(f"{element}-subnet.h5",compile=False)               
        subnet = Sequential()
        subnet.add(globals()[str(element)+"-subnet"]) 
        elemental_subnets[element] = subnet
    return elemental_subnets

In [60]:
def build_molecular_net(at_idx_map, elemental_subnets):
    inputs = []
    subnet_outputs = []
    atomic_nets = {}
    for type, atom_indices in at_idx_map.items():
        atomic_nets[type] = {}
        for idx in atom_indices:
            input = Input(shape=(input_dim,), dtype="float32",
                            name=f"{type}-{idx}-ele")
            inputs.append(input)
            atomic_subnet = elemental_subnets[type](input)
            subnet_outputs.append(atomic_subnet)
            #atomic_nets[type][atom_idx] = atomic_subnet
    total_output = keras.layers.add(subnet_outputs)
    molecular_net = Model(inputs, total_output)
    return molecular_net

In [61]:
def build_molecular_net(at_idx_map, elemental_subnets):
    inputs = []
    subnet_outputs = []
    atomic_nets = {}
    for type, atom_indices in at_idx_map.items():
        atomic_nets[type] = {}
        for idx in atom_indices:
            input = Input(shape=(n_symm_func,), dtype="float32",
                            name=f"{type}-{idx}-ele")
            inputs.append(input)
            atomic_subnet = elemental_subnets[type](input)
            subnet_outputs.append(atomic_subnet)
    total_output = keras.layers.add(subnet_outputs)
    molecular_net = Model(inputs, total_output)
    return molecular_net

In [62]:
subnets = build_subnets(SUPPORTED_ELEMENTS)
model = build_molecular_net(at_idx_map, subnets)

In [63]:
print(model.summary())

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
H-40-ele (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
H-41-ele (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
O-39-ele (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
Au-0-ele (InputLayer)           [(None, 150)]        0                                            
_______________________________________________________________________________________

In [64]:
y_pred_scaled4 = model.predict(inp_train) #Application of the model
y_pred4 = y_pred_scaled4.T[0]  # in kcal/mol unit
print(y_pred4) #kcal/mol units
#np.savetxt("4e", y_pred4, delimiter=",")

[-11.791773    -9.663462    -8.878822     1.313339     6.6674833
   8.327259     8.334852    11.44001     10.209883    11.653681
  13.4632225  -10.261246    -9.6208935   -6.980595     0.526646
   3.3227146    8.515295     9.885639    10.61055     10.61742
   8.457615    10.078167    -4.1210117   -2.1497326    2.1007602
   8.653671     9.507719     9.308023    10.656942    13.416601
  14.017849    11.424498    13.030632     0.94356555  -0.56414163
   3.7035785    5.4054646   10.267338    11.788475    12.96336
  15.0266285   14.13434     12.935993    14.579935    -8.38172
  -6.0489025   -5.6754937    0.1221628    6.186049     8.364603
   9.310431    10.769949    11.798788    15.241671    16.125538
  -3.632282    -5.2635407    0.60769963   4.7542887   15.794901
  14.772625    13.939072    13.227376    11.120124    14.017834
  14.374647     4.319134     4.162554     4.2642493   11.6196575
  13.919523    14.220432     9.306374     6.158385     4.988257
  13.608097    16.156172  ]


In [66]:
y_pred4*0.043 #ev units
np.savetxt("4eV", y_pred4*0.043, delimiter=",")